In [1]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from dotenv import load_dotenv
import os

## setting the environment variables
env_path= r"D:\common_credentials\.env"
load_dotenv(dotenv_path=env_path)
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

Patching name='__init__', member=<function LLMLingua.__init__ at 0x0000025EBD2F22A0>, patched=<function function.__call__ at 0x0000025EBD2F2200>
Patching name='compress_text', member=<function LLMLingua.compress_text at 0x0000025EBD2F2340>, patched=<function function.__call__ at 0x0000025EBD2F2520>


In [2]:
#api parameters: https://microsoft.github.io/autogen/0.2/docs/topics/non-openai-models/cloud-togetherai#api-parameters
config_list =[{'model': 'deepseek-r1-distill-llama-70b',
               'api_key': GROQ_API_KEY,
               "api_type": "groq"}]

 ## LLM Config
llm_config = {"config_list": config_list}


# https://microsoft.github.io/autogen/0.2/docs/topics/llm_configuration
deepseek_config = {
    "cache_seed" : 45,  #to get reproducible results always
    "temperaturre": 0.5,
    "config_list": config_list,
    "timeout": 120
}

In [3]:
task = """
**Task**: As an architech, you are required to desing solutiomn for the following business requirement:
    - Data storage for the massive amount of Healthcare IoT data
    - Real time data analytics and scalable machine learning pipeline
    - Scalability
    - Cost Optimization
    - Region pairs in Asia and Europe for disaster recovery
    - Tools for monitoring & obeservability of Machine learning applications
    - Timeline- 6 months
"""

cloud_prompt= """
**Role**: You are a cloud architect at a healthcare company. You are tasked with designing a solution for the following business requirements:
only use open-source frameworks that are popular and have lots of active contributors.
At the end briefly state the advantages of open-source adoption and summarize your solution using table for clarity."""

cloud_prompt= cloud_prompt + task
# print(cloud_prompt)

oss_prompt = """
**Role**: You are an expert open-source sofware advocate. You need to develop architecture propoesal without considering cloud solutions.
only use open-source frameworks that are popular and have lots of active contributors.
At the end briefly state the advantages of open-source adoption and summarize your solution using table for clarity"""
oss_prompt = oss_prompt + task

# print("\n",oss_prompt)

lead_prompt = """
**Role**: You are an expert lead architech tasked with managing a conversation between the cloud and open-source architech. 
Each architech will perform a task and reponds with their results. You will critically review the results and ask for, disadvanatages on their solutions.
You will review each result, and choose the best solution for the company accordance with business requirements and architecure best practice.
You will use any number of summary tables to communicate your decision."""
lead_prompt = lead_prompt + task
# print("\n",lead_prompt)

In [4]:

# https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/user_proxy_agent/
user = UserProxyAgent(name="Supervisor",
                system_message= "A human head of Architecture team",
                code_execution_config= {
                    "last_n_messages": 2,
                    "work_dir": "groupchat",
                    "use_docker":False,
                    "human_input_mode": "Never"
                                })

#cloud agent
cloud_agent = AssistantAgent(name="Cloud Architech",
                             system_message= cloud_prompt,
                             llm_config= deepseek_config,
                            
                             ) 


#open-source sofware advocate
os_advocate = AssistantAgent(name="open-source sofware advocate",
                             system_message= oss_prompt,
                             llm_config= deepseek_config,
                            
                             ) 


#open-source sofware advocate
lead_architech = AssistantAgent(name="lead architech of team",
                             system_message= lead_prompt,
                             llm_config= deepseek_config,
                            
                             )

# https://microsoft.github.io/autogen/0.2/blog/2024/02/11/FSM-GroupChat/
def state_transition(last_speaker, groupchat):
    ## user >> cloud_agent >> os_advocate >> lead_architech >> None
    if last_speaker is user:
        return cloud_agent
    elif last_speaker is cloud_agent:
        return os_advocate
    elif last_speaker is os_advocate:
        return lead_architech
    elif last_speaker is lead_architech:
        #lead > end
        return None

In [5]:
# https://microsoft.github.io/autogen/0.2/docs/notebooks/agentchat_groupchat/
groupchat= GroupChat(
    agents=[user, cloud_agent, os_advocate, lead_architech],
    messages=[],
    max_round=6,
    speaker_selection_method= state_transition   # "auto": the next speaker is selected automatically by LLM. "manual": the next speaker is selected manually by user input."random": the next speaker is selected randomly.
)

In [6]:
#GroupChatManager: A chat manager agent that can manage a group chat of multiple agents.
# https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/groupchat/#groupchatmanager

manager= GroupChatManager(groupchat=groupchat,
                        llm_config= llm_config)


response = user.initiate_chat(
    manager,
    message= "Provide your best architecture based on these business requirements",
    silent= True #If True, the agent will not print any messages to the console.
)


Next speaker: Cloud Architech

Cloud Architech (to chat_manager):

<think>
Okay, so I'm trying to design a solution for a healthcare company's IT infrastructure based on the given business requirements. Let me break this down step by step. 

First, I need to handle massive amounts of IoT data storage. Healthcare IoT devices generate a lot of data, probably in various formats. So, the storage solution needs to handle structured, semi-structured, and unstructured data efficiently. I've heard about Apache HBase and Apache Cassandra being good for NoSQL and handling large-scale data. HBase is column-oriented and works well with Hadoop, which might be useful for analytics. Cassandra is also great for high availability and scalability, which seems important here.

Next, real-time data analytics and a scalable machine learning pipeline are needed. Apache Kafka comes to mind for real-time data streaming. It can handle high-throughput and provides low-latency data processing, which is perfect 

c:\Users\Dell\.conda\envs\autogen\Lib\site-packages\autogen\oai\groq.py:282: UserWarning: Cost calculation not available for model deepseek-r1-distill-llama-70b
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


open-source sofware advocate (to chat_manager):



### Open-Source Architecture Proposal for Healthcare IoT Data Solution

#### **Business Requirements:**
1. **Data Storage** for massive amounts of Healthcare IoT data
2. **Real-time Data Analytics** and a **scalable Machine Learning pipeline**
3. **Scalability**
4. **Cost Optimization**
5. **Disaster Recovery** across Asia and Europe regions
6. **Monitoring & Observability** of Machine Learning applications
7. **Timeline**: 6 months

---

### **Architecture Proposal**

#### **1. Data Storage**
- **Massive Data Storage Needs**: Healthcare IoT devices generate vast amounts of data, including structured, semi-structured, and unstructured data.
- **Solution**:
  - **Apache HBase**: A distributed, column-oriented database built on top of Hadoop Distributed File System (HDFS). It is ideal for large-scale structured and semi-structured data storage.
  - **Apache Cassandra**: A highly available, distributed NoSQL database designed to handle la

In [7]:
type(response)

autogen.agentchat.chat.ChatResult

In [8]:
response

ChatResult(chat_id=None, chat_history=[{'content': 'Provide your best architecture based on these business requirements', 'role': 'assistant', 'name': 'Supervisor'}, {'content': "<think>\nOkay, so I'm trying to design a solution for a healthcare company's IT infrastructure based on the given business requirements. Let me break this down step by step. \n\nFirst, I need to handle massive amounts of IoT data storage. Healthcare IoT devices generate a lot of data, probably in various formats. So, the storage solution needs to handle structured, semi-structured, and unstructured data efficiently. I've heard about Apache HBase and Apache Cassandra being good for NoSQL and handling large-scale data. HBase is column-oriented and works well with Hadoop, which might be useful for analytics. Cassandra is also great for high availability and scalability, which seems important here.\n\nNext, real-time data analytics and a scalable machine learning pipeline are needed. Apache Kafka comes to mind for

In [9]:
response.chat_history, len(response.chat_history)

([{'content': 'Provide your best architecture based on these business requirements',
   'role': 'assistant',
   'name': 'Supervisor'},
  {'content': "<think>\nOkay, so I'm trying to design a solution for a healthcare company's IT infrastructure based on the given business requirements. Let me break this down step by step. \n\nFirst, I need to handle massive amounts of IoT data storage. Healthcare IoT devices generate a lot of data, probably in various formats. So, the storage solution needs to handle structured, semi-structured, and unstructured data efficiently. I've heard about Apache HBase and Apache Cassandra being good for NoSQL and handling large-scale data. HBase is column-oriented and works well with Hadoop, which might be useful for analytics. Cassandra is also great for high availability and scalability, which seems important here.\n\nNext, real-time data analytics and a scalable machine learning pipeline are needed. Apache Kafka comes to mind for real-time data streaming. It

In [10]:
from pprint import pprint

pprint(response.chat_history)

[{'content': 'Provide your best architecture based on these business '
             'requirements',
  'name': 'Supervisor',
  'role': 'assistant'},
 {'content': '<think>\n'
             "Okay, so I'm trying to design a solution for a healthcare "
             "company's IT infrastructure based on the given business "
             'requirements. Let me break this down step by step. \n'
             '\n'
             'First, I need to handle massive amounts of IoT data storage. '
             'Healthcare IoT devices generate a lot of data, probably in '
             'various formats. So, the storage solution needs to handle '
             'structured, semi-structured, and unstructured data efficiently. '
             "I've heard about Apache HBase and Apache Cassandra being good "
             'for NoSQL and handling large-scale data. HBase is '
             'column-oriented and works well with Hadoop, which might be '
             'useful for analytics. Cassandra is also great for high 

In [11]:
for i in response.chat_history:
    print(i['name'])
    pprint(i['content'])
    print("\n")

Supervisor
'Provide your best architecture based on these business requirements'


Cloud Architech
('<think>\n'
 "Okay, so I'm trying to design a solution for a healthcare company's IT "
 'infrastructure based on the given business requirements. Let me break this '
 'down step by step. \n'
 '\n'
 'First, I need to handle massive amounts of IoT data storage. Healthcare IoT '
 'devices generate a lot of data, probably in various formats. So, the storage '
 'solution needs to handle structured, semi-structured, and unstructured data '
 "efficiently. I've heard about Apache HBase and Apache Cassandra being good "
 'for NoSQL and handling large-scale data. HBase is column-oriented and works '
 'well with Hadoop, which might be useful for analytics. Cassandra is also '
 'great for high availability and scalability, which seems important here.\n'
 '\n'
 'Next, real-time data analytics and a scalable machine learning pipeline are '
 'needed. Apache Kafka comes to mind for real-time data stream

In [12]:
import pandas as pd

In [13]:
pd.DataFrame(response.chat_history)

,content,role,name
0,Provide your best architecture based on these ...,assistant,Supervisor
1,"<think>\nOkay, so I'm trying to design a solut...",user,Cloud Architech
2,\n\n### Open-Source Architecture Proposal for ...,user,open-source sofware advocate
3,\n\n**Critical Review and Analysis of the Open...,user,lead architech of team
